In [1]:
import os
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import numpy as np
import pandas as pd
from konlpy.tag import Okt
import random
import re

In [2]:
import os
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import numpy as np
import pandas as pd
from konlpy.tag import Okt
from typing import List
from fastapi import FastAPI
from fastapi.responses import JSONResponse
from pydantic import BaseModel
import os
import time
import random
import dijkstra
import pymysql
import tags

# 둘중 하나만 사용

# file path
###################################################################################

current_dir = os.getcwd()
# df_path = os.path.join(current_dir, 'travel_spot_v1.csv') sql로 변경

model_path = os.path.join(current_dir, 'travel_model_v5.pkl')
info_path = os.path.join(current_dir, 'DB_INFO.pkl')

###################################################################################

with open(info_path, "rb") as file:
    USER = pickle.load(file)
    PASSWD = pickle.load(file)
    HOST = pickle.load(file)
    PORT = pickle.load(file)
    NAME = pickle.load(file)

# pymysql로 dataFrame 호출

host = HOST
user = USER
password = PASSWD
database = NAME
charset = 'utf8mb4'

conn = pymysql.connect(host=host,user=user,password=password,database=database,charset=charset)
sql = "select * from TRAVEL_SPOT"
cursor = conn.cursor()
cursor.execute(sql)
db = pd.DataFrame(cursor.fetchall(), columns= [i[0] for i in cursor.description])
db.to_csv('df.csv')
df = pd.read_csv('./df.csv', indexcol=0)

TypeError: read_csv() got an unexpected keyword argument 'indexcol'

In [ ]:
pd.read_csv('./df.csv', index_col=)

In [6]:
# 현재 스크립트의 디렉토리 가져오기
current_directory = os.getcwd()

# 한 단계 상위 디렉토리로 이동
parent_directory = os.path.dirname(current_directory)
model = os.path.join(current_directory,'DB_INFO.pkl')
print(f"현재 스크립트의 디렉토리: {current_directory}")
print(f"한 단계 상위 디렉토리: {parent_directory}")
print(model)

현재 스크립트의 디렉토리: /home/doxg/code/DP_Service/python
한 단계 상위 디렉토리: /home/doxg/code/DP_Service
/home/doxg/code/DP_Service/python/DB_INFO.pkl


In [1]:
import os
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import numpy as np
import pandas as pd
from konlpy.tag import Okt
from typing import List
from fastapi import FastAPI
from fastapi.responses import JSONResponse
from pydantic import BaseModel
import os
import time
import random
import dijkstra
import pymysql
import tags


current_directory = os.getcwd()
model = os.path.join(current_directory,'DB_INFO.pkl')

with open(model, "rb") as file:
    USER = pickle.load(file)
    PASSWD = pickle.load(file)
    HOST = pickle.load(file)
    PORT = pickle.load(file)
    NAME = pickle.load(file)

# pymysql로 dataFrame 호출

host = HOST
user = USER
password = PASSWD
database = NAME
charset = 'utf8mb4'

conn = pymysql.connect(host=host,user=user,password=password,database=database,charset=charset)
sql = "select * from TRAVEL_SPOT"
cursor = conn.cursor()
cursor.execute(sql)
df = pd.DataFrame(cursor.fetchall(), columns= [i[0] for i in cursor.description])
cursor.execute(sql)
df2 = pd.DataFrame(cursor.fetchall(), columns= [i[0] for i in cursor.description])
df = df.fillna('')
df2 = df2.fillna('')

In [2]:
df.head(1)

,SPOT_ID,SPOT_CITY,SPOT_CITY_CODE,SPOT_CITY_CONTENT_TYPE,SPOT_TITLE,SPOT_CATCHTITLE,SPOT_OVERVIEW,SPOT_TREATMENU,SPOT_CONLIKE,SPOT_CONREAD,SPOT_CONSHARE,SPOT_IMGPATH,SPOT_ADDR,SPOT_INFOCENTER,SPOT_PARKING,SPOT_USETIME,SPOT_TAGNAME,SPOT_DETAIL,SPOT_LON,SPOT_LAT
0,125266,강원,32,12,국립 청태산자연휴양림,인공림과 자연림의 조화,"해발 1,200m의 청태산을 주봉으로 하여 인공림과 천연림이 잘 조화된 울창한 산림...",,6,7410,7,https://cdn.visitkorea.or.kr/img/call?cmd=VIEW...,강원특별자치도 횡성군 둔내면 청태산로 610,033-343-9707,있음,[숙박시설] 15:00~익일11:00<br>\n[일일개장] 09:00~18:00<br>,"가족여행,강원권,강원도_여행지_추천,걷기길,관광지,국립청태산자연휴양림,등산,등산로,...",내국인 예약안내:인터넷 예약,128.2919115,37.5225141


SPOT_ID 
SPOT_CITY
SPOT_CITY_CODE
SPOT_CITY_CONTENT_TYPE
SPOT_TITLE 
SPOT_CATCHTITLE 
SPOT_OVERVIEW
SPOT_TREATMENU
SPOT_CONLIKE 
SPOT_CONREAD 
SPOT_CONSHARE 
SPOT_IMGPATH
SPOT_ADDR 
SPOT_INFOCENTER
SPOT_PARKING
SPOT_USETIME
SPOT_TAGNAME
SPOT_DETAIL 
SPOT_LON 
SPOT_LAT
     

In [8]:
df2['SPOT_CITY'] = '전체'

In [9]:
joined_df = pd.concat([df,df2], axis=0, ignore_index=True)

In [10]:
len(df),len(df2),len(joined_df)

(10638, 10638, 21276)

In [11]:
joined_df['vec'] = joined_df['SPOT_CATCHTITLE'].apply(lambda x: x[:100]) + joined_df['SPOT_TREATMENU'].apply(lambda x: x[:200]) + joined_df['SPOT_TAGNAME'].apply(lambda x: x[:200]) + joined_df['SPOT_OVERVIEW'].apply(lambda x: x[:200])
joined_df['vec_f'] = joined_df['SPOT_CATCHTITLE'].apply(lambda x: x[:100]) + joined_df['SPOT_TREATMENU'].apply(lambda x: x[:200]) + joined_df['SPOT_TAGNAME'].apply(lambda x: x[:200])

In [12]:
spot_df = joined_df[joined_df['SPOT_CITY_CONTENT_TYPE'] != 39]
food_df = joined_df[joined_df['SPOT_CITY_CONTENT_TYPE'] == 39]
spot_df.__len__(), food_df.__len__()

(9416, 11860)

In [22]:
len(spot_df), len(food_df)

(9416, 11860)

In [13]:
okt = Okt()

In [14]:
# spot 학습

vectorizer_spot = TfidfVectorizer()
vectorizer_food = TfidfVectorizer()



# 전체 도시에 대한 TF-IDF 행렬 계산
tag_list_all = []
for i in spot_df[spot_df['SPOT_CITY'] != '전체']['vec'].tolist():
    tag_list_all.append(' '.join(list(set([j for j in okt.nouns(i) if len(j) > 1 and any(char.isalpha() and not char.isascii() for char in j)]))))
    # print(' '.join(list(set([j for j in okt.nouns(i) if len(j) > 1 and any(char.isalpha() and not char.isascii() for char in j)]))))
    # print(len(tag_list_all))

matrix_spot = vectorizer_spot.fit_transform(tag_list_all)

# 각 도시에 대한 TF-IDF 행렬 계산
city_matrices_spot = {}
for city in spot_df['SPOT_CITY'].unique():
    tag_list_city = []
    city_df = spot_df[spot_df['SPOT_CITY'] == city]
    
    for i in city_df['vec'].tolist():
        tag_list_city.append(' '.join(list(set([j for j in okt.nouns(i) if len(j) > 1 and any(char.isalpha() and not char.isascii() for char in j)]))))
        # print(' '.join(list(set([j for j in okt.nouns(i) if len(j) > 1 and any(char.isalpha() and not char.isascii() for char in j)]))))
        # print(tag_list_city[-1][-5:-1])
        

    city_matrices_spot[city] = vectorizer_spot.transform(tag_list_city)


In [15]:
# food 학습

# 전체 도시에 대한 TF-IDF 행렬 계산
tag_list_all_food = []
for i in food_df[food_df['SPOT_CITY'] != '전체']['vec'].tolist():
    tag_list_all_food.append(' '.join(list(set([j for j in okt.nouns(i) if len(j) > 1 and any(char.isalpha() and not char.isascii() for char in j)]))))
    # print(' '.join(list(set([j for j in okt.nouns(i) if len(j) > 1 and any(char.isalpha() and not char.isascii() for char in j)]))))
    # print(len(tag_list_all_food))


matrix_food = vectorizer_food.fit_transform(tag_list_all_food)

# 각 도시에 대한 TF-IDF 행렬 계산
city_matrices_food = {}
for city in food_df['SPOT_CITY'].unique():
    tag_list_city_food = []
    city_df = food_df[food_df['SPOT_CITY'] == city]
    
    for i in city_df['vec'].tolist():
        tag_list_city_food.append(' '.join(list(set([j for j in okt.nouns(i) if len(j) > 1 and any(char.isalpha() and not char.isascii() for char in j)]))))
        # print(' '.join(list(set([j for j in okt.nouns(i) if len(j) > 1 and any(char.isalpha() and not char.isascii() for char in j)]))))
        print(tag_list_city_food[-1][-5:-1])
        
        
    city_matrices_food[city] = vectorizer_food.transform(tag_list_city_food)


전통 산
육회 전
판매 사
한식 곳
무리 구
면 가이
정성 전
민국 국
전통 소
보이 연
이트 재
특별 수
명맥 정
통 떡볶
비지 전
수 한아
물김 천
지금 전
다소 자
기와 천
 스파게
무 식당
구입 루
고급 세
전통 국
일구 천
정성 전
운지 별
천구 전
정성 당
냉면 전
전통 연
램 압구
영화 전
비 강남
대방 여
크림 카
국물 진
문학 전
묵 떡볶
제보 국
영업 개
카페 공
성 통나
전통 세
조화 소
이드 주
 대한민
동구 서
낙지 전
통 뚝배
게티 적
정성 전
귀빈 설
캘리포니
석식 매
공감 세
통 막국
비 세꼬
비 세꼬
소금 소
카페 염
 대한민
도로 비
코스 안
갈비 반
팔경 미
이프 카
카페 대
 아이치
업 매스
우스 카
춘천 오
더욱 등
즉석 차
국물 주
국수 간
내공 전
코스 개
방탄소년
이크 카
식감 도
닭발 생
크 청담
야외 애
나무 훈
추억 다
전통 소
고수 호
국물 과
철판 호
이크 메
인기 본
리역 호
철판 도
밥상 호
등뼈 생
별 삼청
뼈 뚝배
고급 소
소스 주
포즈 오
예 돈까
웨딩 카
 와이키
통 해물
이크 유
페 라운
호선 소
통 오소
카페 쇼
구이 카
급 떡갈
목살 타
실역 호
직원 호
터디 카
카페 자
페 베이
이드 경
카페 별
대립 더
영자 카
이트 카
도심 카
전망 카
야외 카
조화 메
고수 카
카페 제
소스 제
백숙 솥
민국 훈
묵 떡볶
동 갈비
전통 목
야외 카
거 명일
울대 쇼
만두 호
구비 카
크림 전
사용 은
호선 전
 대한민
품 뚝배
호선 카
스 창덕
호선 카
호선 소
선 뚝배
호선 소
별로 훠
사 양천
로동 레
돈암 호
호선 전
호선 카
호선 국
돈비 국
호선 전
앤찜 호
호선 제
이크 소
이드 차
이역 국
본점 호
물 수상
원정 호
우럭 호
육회 명
보샤 전
구이 호
고층 광
고급 수
호선 전
념장 호
스 떡볶
날 뚝배
닭칼 호
 장한평
남구 서
담동 광
성산 어
전체 카
볶이 순
호선 차
잎 떡볶
페 핫도
접근 조
선 팀호
솥밥 천
국수 차
냉면 전
호선 천
뼈 뚝배
호선 솥
서기 포
와룡 자
철판 닭


In [ ]:
os.p

In [16]:
# 저장
with open('travel_model_v5.pkl', 'wb') as file:
        pickle.dump({"spot":{
                "vectorizer": vectorizer_spot,
                "matrix": matrix_spot,
                "city_matrices": city_matrices_spot
        },"food":{
                "vectorizer": vectorizer_food,
                "matrix": matrix_food,
                "city_matrices": city_matrices_food
        }}, file)

In [26]:
# 불러오기
with open('./travel_model_v3.pkl', 'rb') as file:
    loaded_data = pickle.load(file)
    
    vectorizer_spot = loaded_data["spot"]["vectorizer"]
    matrix_spot = loaded_data["spot"]["matrix"]
    city_matrices_spot = loaded_data["spot"]["city_matrices"]

    vectorizer_food = loaded_data["food"]["vectorizer"]
    matrix_food = loaded_data["food"]["matrix"]
    city_matrices_food = loaded_data["food"]["city_matrices"]

In [ ]:
area = '인천'
question = '캠핑'

df = spot_df[spot_df['city']== f'{area}']

question_tfidf = vectorizer_spot.transform(okt.morphs(f'{question}'))

cos_similarities = cosine_similarity(question_tfidf, city_matrices_spot[f'{area}'])
sorted_indices = np.argsort(cos_similarities[0])[::-1][:5]
print('indices= ',len(sorted_indices))
print('df= ', len(df))
# 식당인 것, 아닌 

index = sorted_indices[0]



print(
str(df.iloc[index]['id']),
str(df.iloc[index]['city']),
str(df.iloc[index]['title']),
float(cos_similarities[0][index]),
str(df.iloc[index]['catchtitle']))



In [15]:
okt = Okt()
a = "샥스핀, 식사류해삼류,단품 요리 바닷가재 등  짬뽕 등  등"
b = "샥스핀, 해삼류, 바닷가재 등 단품 요리, 짜장면, 짬뽕 등 식사류 등"
A = [i for i in okt.morphs(a) if len(i) >1]
B = [i for i in okt.morphs(b) if len(i) >1]
vectorizer = TfidfVectorizer()
text = vectorizer_spot.transform(A)

cosine_similarities = cosine_similarity(text, matrix_spot)
cosine_similarities[0]

print(random.sample(A, len(A)))

['요리', '식사', '샥스핀', '짬뽕', '해삼류', '바닷가재']


In [38]:
len(spot_df), len(food_df), len(df3)

(9268, 11766, 21034)

In [82]:
tag_list = [' '.join([j for j in okt.morphs(i) if len(j) > 1]) for i in spot_df['tagName'].tolist()[:2]]
tag_list

['가족 여행 걷기길 관광지 데이트 코스 반려동물 여행지 생태관광 서울숲 수도권 수도권 반려동물 여행지 아이 함께 연인 함께 연중 피크닉 한국 관광 100 휴식 공간 휴식 여행 휴식 하기 휴식 하기 좋은',
 '관광지 길상사 서울 ), 불교 불교 문화 사찰 사찰 여행 산사 수도권 역사 연중 연속 으로 자연 좋은 자연환경 전통 사찰 종교 한국 불교 휴식 공간 휴식 여행 휴식 하기 휴식 하기 좋은']

In [39]:
vectorizer_spot = TfidfVectorizer()

tag_list = [' '.join([j for j in okt.morphs(i) if len(j) > 1]) for i in spot_df['tagName'].tolist()]
matrix_spot = vectorizer_spot.fit_transform(tag_list)

# 각 지역에 대한 TF-IDF 행렬 계산
city_matrices_spot = {}    
for city in spot_df['city'].unique():
    city_df = spot_df[spot_df['city'] == city]
    city_tag_list = [' '.join([j for j in okt.morphs(i) if len(j) > 1]) for i in city_df['tagName'].tolist()]
    city_matrices_spot[city] = vectorizer_spot.transform(city_tag_list)

In [5]:
vectorizer_food = TfidfVectorizer()

tag_list = [' '.join([j for j in okt.morphs(str(i)) if len(j) > 1]) for i in food_df['tf'].tolist()]
matrix_food = vectorizer_food.fit_transform(tag_list)

# 각 지역에 대한 TF-IDF 행렬 계산
city_matrices_food = {}    
for city in food_df['city'].unique():
    city_df = food_df[food_df['city'] == city]
    city_tag_list = [' '.join([j for j in okt.morphs(str(i)) if len(j) > 1]) for i in city_df['tf'].tolist()]
    city_matrices_food[city] = vectorizer_food.transform(city_tag_list)

In [6]:

# 저장
with open('travel_model_v2.pkl', 'wb') as file:
        pickle.dump({"spot":{
                "vectorizer": vectorizer_spot,
                "matrix": matrix_spot,
                "city_matrices": city_matrices_spot
        },"food":{
                "vectorizer": vectorizer_food,
                "matrix": matrix_food,
                "city_matrices": city_matrices_food
        }}, file)

In [4]:
with open('./travel_model_v2.pkl', 'rb') as file:
    loaded_data = pickle.load(file)
    
    vectorizer_spot = loaded_data["spot"]["vectorizer"]
    matrix_spot = loaded_data["spot"]["matrix"]
    city_matrices_spot = loaded_data["spot"]["city_matrices"]

    vectorizer_food = loaded_data["food"]["vectorizer"]
    matrix_food = loaded_data["food"]["matrix"]
    city_matrices_food = loaded_data["food"]["city_matrices"]

/home/doxg/.pyenv/versions/3.8.18/lib/python3.8/site-packages/sklearn/base.py:313: UserWarning: Trying to unpickle estimator TfidfTransformer from version 1.3.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/home/doxg/.pyenv/versions/3.8.18/lib/python3.8/site-packages/sklearn/base.py:313: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 1.3.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [ ]:
#     # 미리 계산된 해당 지역의 TF-IDF 행렬 사용
# if question.area == '전체':
#     city_tfidf_matrix = tfidf_matrix
# else:
#     city_tfidf_matrix = city_tfidf_matrices.get(question.area)
#     df = df[df['city']== f'{question.area}']

In [35]:
city_matrices_spot['충남']

<351x5323 sparse matrix of type '<class 'numpy.float64'>'
	with 5212 stored elements in Compressed Sparse Row format>

In [74]:
city_matrices_spot['인천']

<5995x18425 sparse matrix of type '<class 'numpy.float64'>'
	with 5995 stored elements in Compressed Sparse Row format>

In [90]:
area = '인천'
question = '캠핑'

df = spot_df[spot_df['city']== f'{area}']

question_tfidf = vectorizer_spot.transform(okt.morphs(f'{question}'))

cos_similarities = cosine_similarity(question_tfidf, city_matrices_spot[f'{area}'])
sorted_indices = np.argsort(cos_similarities[0])[::-1]
print('indices= ',len(sorted_indices))
print('df= ', len(df))
# 식당인 것, 아닌 

index = sorted_indices[0]
print(index)


print(
str(df.iloc[index]['id']),
str(df.iloc[index]['city']),
str(df.iloc[index]['title']),
float(cos_similarities[0][index]),
str(df.iloc[index]['catchtitle']))



NotFittedError: The TF-IDF vectorizer is not fitted

In [70]:
area = '대구'
question = '피자'

df = food_df[food_df['city']== f'{area}']

question_tfidf = vectorizer_food.transform(okt.morphs(f'{question}'))

cos_similarities = cosine_similarity(question_tfidf, city_matrices_food[f'{area}'])
sorted_indices = np.argsort(cos_similarities[0])[::-1][:5]
print('indices= ',len(sorted_indices))
print('df= ', len(df))
# 식당인 것, 아닌 

index = sorted_indices[0]



print(
str(df.iloc[index]['id']),
str(df.iloc[index]['city']),
str(df.iloc[index]['title']),
float(cos_similarities[0][index]),
str(df.iloc[index]['catchtitle']))



indices=  5
df=  192
2866244 대구 루미너스 0.42204924949455175 모던한 분위기의 한우 안심 스테이크 맛집


In [14]:
np.argsort(cos_similarities[0])[::-1][0]

63

In [15]:
cos_similarities[0][63]

0.30805653331899097

In [27]:
df = pd.read_csv('./travel_spot_v1.csv', index_col=0)

In [29]:
okt = Okt()

In [35]:
question_area = '인천'
question_question = '효도여행'

# 미리 계산된 해당 지역의 TF-IDF 행렬 사용
if question_area == '전체':
    city_tfidf_matrix = tfidf_matrix
else:
    city_tfidf_matrix = city_tfidf_matrices.get(question_area)
    df = df[df['city']== f'{question_area}']

print('여기여기')

if city_tfidf_matrix is None:
    print(JSONResponse(content={"error": f"No data found for city: {question_area}"}, status_code=404))

# 질문과 선택된 지역의 TF-IDF로 유사도 계산
question_tfidf = tfidf_vectorizer.transform(okt.morphs(question_question))
cos_similarities = cosine_similarity(question_tfidf, city_tfidf_matrix)
sorted_indices = np.argsort(cos_similarities[0])[::-1]
len(sorted_indices)

여기여기


352

In [81]:
restaurant = sorted_indices[df.iloc[sorted_indices]['contentType'].eq(39)].tolist()[:5]
non_restaurant = sorted_indices[~df.iloc[sorted_indices]['contentType'].eq(39)].tolist()[:5]

restaurant,non_restaurant

([111, 112, 113, 114, 115], [351, 350, 296, 288, 289])

In [62]:
len(df), len(sorted_indices)

(352, 352)

In [60]:
len(sorted_indices)

352

In [52]:

print('여기여기2')

from itertools import islice
# 식당인 것, 아닌 것
restaurant = islice((index for index in sorted_indices if df.iloc[index]['contentType'] == 39 ), 5)
non_restaurant = islice((index for index in sorted_indices if df.iloc[index]['contentType'] != 39 ), 5)

list(restaurant), list(non_restaurant)

여기여기2


([111, 112, 113, 114, 115], [351, 350, 296, 288, 289])

In [51]:
str(df.iloc[111]['city'])

'인천'

In [55]:
str(df.iloc[111]['id'])

'2758144'

In [53]:
def add_to_similar_tags(sorted_indices,similar_tags):
    bag = []
    for index in sorted_indices:
        print("Current index:", index)
        bag.append({
            "id": str(df.iloc[index]['id']),
            "area": str(df.iloc[index]['city'])})
        similar_tags.append(bag)

In [78]:
restaurant


In [57]:

# 결과를 저장할 리스트 초기화
similar_tags = []
print('여기여기 3')
# 리스트로 추가
def add_to_similar_tags(sorted_indices,similar_tags):
    bag = []
    for index in sorted_indices:
        print("Current index:", index)
        bag.append({
            "id": str(df.iloc[index]['id']),
            "area": str(df.iloc[index]['city']),
            "title": str(df.iloc[index]['title']),
            "similarity": float(cos_similarities[0][index]),
            "catchtitle": str(df.iloc[index]['catchtitle']),
            # "detail": str(df.iloc[index]['detail']),
            "treatMenu": str(df.iloc[index]['treatMenu']),
            "tagName": str(df.iloc[index]['tagName']),
            "addr": str(df.iloc[index]['addr']),
            "info": str(df.iloc[index]['info']),
            # "lat": str(df.iloc[index]['parking']),
            "useTime": str(df.iloc[index]['useTime']),
            "conLike": str(df.iloc[index]['conLike']),
            "conRead": str(df.iloc[index]['conRead']),
            "conShare": str(df.iloc[index]['conShare']),
            # "overView": str(df.iloc[index]['overView']),
            "lat": str(df.iloc[index]['lat']),
            "lon": str(df.iloc[index]['lon'])
            
            })
    similar_tags.append(bag)
print('여기여기4')

# 함수를 사용
add_to_similar_tags(non_restaurant,similar_tags)
add_to_similar_tags(restaurant,similar_tags)
similar_tags

여기여기 3
여기여기4


[[], []]

In [76]:
sorted_indices.__len__()

352

In [73]:
# 결과를 저장할 리스트 초기화
similar_tags = []

def add_to_similar_tags(sorted_indices):
    bag = []
    sorted_indices = list(sorted_indices)[:len(df)]
    print("sorted_indices length:", len(sorted_indices))
    for index in sorted_indices:
        print("Processing index:", index)
        if 0 <= index < len(df):
            print("Adding to bag:", index)
            bag.append({
                "id": str(df.iloc[index]['id']),
                "area": str(df.iloc[index]['city']),
                "title": str(df.iloc[index]['title']),
                "similarity": float(cos_similarities[0][index]),
                "catchtitle": str(df.iloc[index]['catchtitle']),
                "treatMenu": str(df.iloc[index]['treatMenu']),
                "tagName": str(df.iloc[index]['tagName']),
                "addr": str(df.iloc[index]['addr']),
                "info": str(df.iloc[index]['info']),
                "useTime": str(df.iloc[index]['useTime']),
                "conLike": str(df.iloc[index]['conLike']),
                "conRead": str(df.iloc[index]['conRead']),
                "conShare": str(df.iloc[index]['conShare']),
                "lat": str(df.iloc[index]['lat']),
                "lon": str(df.iloc[index]['lon'])
            })
        else:
            print("Index out of bounds:", index)
    print("Adding bag to similar_tags:", bag)
    similar_tags.append(bag)

# 함수를 사용
add_to_similar_tags(non_restaurant)
add_to_similar_tags(restaurant)

print("Final similar_tags:", similar_tags)


sorted_indices length: 0
Adding bag to similar_tags: []
sorted_indices length: 0
Adding bag to similar_tags: []
Final similar_tags: [[], []]


In [15]:
import pymysql

host = 'database-1.ckodcngpvqn9.ap-northeast-2.rds.amazonaws.com'
user = 'admin'
password = 'zlakzkfp'
database = 'sam_db'
charset = 'utf8mb4'

conn = pymysql.connect(host=host,user=user,password=password,database=database,charset=charset)

sql = "select * from TRAVEL_SPOT"
cursor = conn.cursor()
cursor.execute(sql)
df = pd.DataFrame(cursor.fetchall(), columns= [i[0] for i in cursor.description])

,SPOT_ID,SPOT_CITY,SPOT_CITY_CODE,SPOT_CITY_CONTENT_TYPE,SPOT_TITLE,SPOT_CATCHTITLE,SPOT_OVERVIEW,SPOT_TREATMENU,SPOT_CONLIKE,SPOT_CONREAD,SPOT_CONSHARE,SPOT_IMGPATH,SPOT_ADDR,SPOT_INFOCENTER,SPOT_PARKING,SPOT_USETIME,SPOT_TAGNAME,SPOT_DETAIL,SPOT_LON,SPOT_LAT
0,125266,전체,32,12,국립 청태산자연휴양림,인공림과 자연림의 조화,"해발 1,200m의 청태산을 주봉으로 하여 인공림과 천연림이 잘 조화된 울창한 산림...",,6,7396,7,https://cdn.visitkorea.or.kr/img/call?cmd=VIEW...,강원특별자치도 횡성군 둔내면 청태산로 610,033-343-9707,있음,[숙박시설] 15:00~익일11:00<br>\n[일일개장] 09:00~18:00<br>,"가족여행,강원권,강원도_여행지_추천,걷기길,관광지,국립청태산자연휴양림,등산,등산로,...",내국인 예약안내:인터넷 예약,128.2919115,37.5225141
1,125405,전체,35,12,토함산자연휴양림,불국사와 석굴암을 품고 있는 토함산의 휴양시설,토함산 자연휴양림은 삼국통일의 위업을 달성한 신라천년의 찬란한 문화유적을 간직한 국...,,0,1839,2,https://cdn.visitkorea.or.kr/img/call?cmd=VIEW...,경상북도 경주시 양북면 불국로 1208-45,054-750-8700,주차 가능,[숙박]<br/>\n- 입실 15:00~20:00 / 퇴실 12:00<br/>\n[...,"관광지,자연",내국인 예약안내:홈페이지 예약 가능,129.3655037,35.7619577
2,125406,전체,4,12,비슬산자연휴양림,자연경관 그대로를 활용한 휴식공간,비슬산 자연 휴양림은 자연경관이 수려하고 기암괴석이 풍부한 비술산 내에 자리 잡은 ...,,4,2248,0,https://cdn.visitkorea.or.kr/img/call?cmd=VIEW...,대구광역시 달성군 유가읍 일연선사길 99,053-659-4400~1,"있음(공영주차장, 사설주차장)<br />\n※ 야영객 차량은 출입이 불가능, 휴양림...","상시개방(단, 시설 이용 시는 입, 퇴실시간 적용)<br>\n[숙박시설] 당일 15...","가족여행,걷기길,경상권,관광지,등산,등산로,비슬산,비슬산자연휴양림,여름방학가볼만한곳...",내국인 예약안내:홈페이지 예약,128.5129249,35.6900527
3,125407,전체,35,12,불정자연휴양림,레포츠와 휴식을 동시에 즐길 수 있는 불정자연휴양림,불정자연휴양림은 경북 문경의 불정동 마을 안쪽에 위치한 나트막한 봉우리인 수정봉 (...,,0,1478,1,https://cdn.visitkorea.or.kr/img/call?cmd=VIEW...,경상북도 문경시 불정길 180,불정자연휴양림 054-552-9443<br>\n문경관광진흥공단 054-553-310...,주차가능,[숙박시설]<br/>\n입실 14:00/퇴실 11:00,"관광지,불정자연휴양림,자연,자연휴양림,힐링여행","내국인 예약안내:인터넷 예약제이며, 당일 이용일 경우 전화예약만 가능<br />\n...",128.1342659,36.6188262
4,125408,전체,35,12,청송 자연휴양림 퇴적암층 (청송 국가지질공원),퇴적 구조를 관찰할 수 있는 학습장,청송자연휴양림 퇴적암층은 상쾌한 공기와 수려한 경관을 모두 갖춘 지질명소이다. 다양...,,4,1528,1,https://cdn.visitkorea.or.kr/img/call?cmd=VIEW...,경상북도 청송군 부남면 청송로 3478-96,청송군 문화관광과 054-870-6111<br>\n청송자연휴양림 054-872-3163,가능,09:00~18:00,"관광지,국가지질공원,자연,자연관광지,자연발생,자연속으로,자연여행,자연좋은곳,자연풍경...","시설이용료:[숲속의 집] 비수기 42,000원~84,000원 / 성수기 70,000...",129.0537206,36.3179194
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10612,3029995,전체,35,12,송도솔밭도시숲,,송도솔밭도시숲은 포항시에서 도시화로 사라져가는 소나무를 복원하여 시민들에게는 휴식공...,,0,32,0,https://cdn.visitkorea.or.kr/img/call?cmd=VIEW...,경상북도 포항시 남구 송도해안길 141 (송도동),054-270-8282,있음(공영 주차장)<br>무료,,관광지,한국어 안내서비스:가능,129.3741449,36.0417179
10613,3035809,전체,31,12,안산다문화음식거리,,안산역 1번 출구에서 약 600m 앞에는 안산다문화마을특구가 있다. 안산시는 세계 ...,,1,52,2,https://cdn.visitkorea.or.kr/img/call?cmd=VIEW...,경기도 안산시 단원구 원곡동,070-4740-2580(안산시관광안내소),가능(외국인주민센터 앞 공영주차장)<br>요금(기본 30분 300원 / 추가 10분...,매장별로 상이함,관광지,한국어 안내서비스:가능,126.7904142,37.3290507
10614,3035874,전체,31,12,대부해솔길 1코스,,대부해솔길은 해안선을 따라 대부도를 한 바퀴 둘러볼 수 있는 산책길로 91km에 이...,,0,24,0,https://cdn.visitkorea.or.kr/img/call?cmd=VIEW...,경기도 안산시 단원구 대부북동,1899-1720(대부도 관광안내소),가능<br>요금(무료),,레포츠,화장실:있음,126.5546122,37.2761527
10615,3036485,전체,34,12,월영산 출렁 다리,,"월영산 출렁다리는 월영산과 부엉산 사이를 잇는 높이 45m, 길이 275m, 폭 1...",,1,17,0,https://cdn.visitkorea.or.kr/img/call?cmd=VIEW...,충청남도 금산군 제원면 천내리,041-754-3837,가능,하절기(3월~10월) 09:00~18:00<br>\n동절기(11월~2월) 09:00...,관광지,입 장 료:무료,127.5975943,36.1152768


In [5]:
result

10617

In [17]:
import pymysql
pymysql.__version__

'1.4.6'

In [14]:
sql = "select * from TRAVEL_SPOT"

cursor = conn.cursor()

result = cursor.execute(sql)

df = pd.DataFrame(cursor.fetchall(), columns= [i[0] for i in cursor.description])

In [6]:
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import numpy as np
import pandas as pd
from konlpy.tag import Okt
from typing import List
from fastapi import FastAPI
from fastapi.responses import JSONResponse
from pydantic import BaseModel
import os
import time
import random
import dijkstra
import pymysql

# 둘중 하나만 사용

# file path
###################################################################################


# current_dir = os.path.dirname(os.path.abspath(__file__))
# df_path = os.path.join(current_dir, 'travel_spot_v1.csv') sql로 변경

# model_path = os.path.join(current_dir, 'travel_model_v4.pkl')
# info_path = os.path.join(current_dir, 'DB_INFO.pkl')

###################################################################################

with open('./DB_INFO.pkl', "rb") as file:
    USER = pickle.load(file)
    PASSWD = pickle.load(file)
    HOST = pickle.load(file)
    PORT = pickle.load(file)
    NAME = pickle.load(file)

# pymysql로 dataFrame 호출

host = HOST
user = USER
password = PASSWD
database = NAME
charset = 'utf8mb4'

conn = pymysql.connect(host=host,user=user,password=password,database=database,charset=charset)
sql = "select * from TRAVEL_SPOT"
cursor = conn.cursor()
cursor.execute(sql)
df = pd.DataFrame(cursor.fetchall(), columns= [i[0] for i in cursor.description])

In [9]:
df

,SPOT_ID,SPOT_CITY,SPOT_CITY_CODE,SPOT_CITY_CONTENT_TYPE,SPOT_TITLE,SPOT_CATCHTITLE,SPOT_OVERVIEW,SPOT_TREATMENU,SPOT_CONLIKE,SPOT_CONREAD,SPOT_CONSHARE,SPOT_IMGPATH,SPOT_ADDR,SPOT_INFOCENTER,SPOT_PARKING,SPOT_USETIME,SPOT_TAGNAME,SPOT_DETAIL,SPOT_LON,SPOT_LAT
0,125266,강원,32,12,국립 청태산자연휴양림,인공림과 자연림의 조화,"해발 1,200m의 청태산을 주봉으로 하여 인공림과 천연림이 잘 조화된 울창한 산림...",,6,7410,7,https://cdn.visitkorea.or.kr/img/call?cmd=VIEW...,강원특별자치도 횡성군 둔내면 청태산로 610,033-343-9707,있음,[숙박시설] 15:00~익일11:00<br>\n[일일개장] 09:00~18:00<br>,"가족여행,강원권,강원도_여행지_추천,걷기길,관광지,국립청태산자연휴양림,등산,등산로,...",내국인 예약안내:인터넷 예약,128.2919115,37.5225141
1,125405,경북,35,12,토함산자연휴양림,불국사와 석굴암을 품고 있는 토함산의 휴양시설,토함산 자연휴양림은 삼국통일의 위업을 달성한 신라천년의 찬란한 문화유적을 간직한 국...,,0,1847,2,https://cdn.visitkorea.or.kr/img/call?cmd=VIEW...,경상북도 경주시 양북면 불국로 1208-45,054-750-8700,주차 가능,[숙박]<br/>\n- 입실 15:00~20:00 / 퇴실 12:00<br/>\n[...,"관광지,자연",내국인 예약안내:홈페이지 예약 가능,129.3655037,35.7619577
2,125406,대구,4,12,비슬산자연휴양림,자연경관 그대로를 활용한 휴식공간,비슬산 자연 휴양림은 자연경관이 수려하고 기암괴석이 풍부한 비술산 내에 자리 잡은 ...,,4,2257,0,https://cdn.visitkorea.or.kr/img/call?cmd=VIEW...,대구광역시 달성군 유가읍 일연선사길 99,053-659-4400~1,"있음(공영주차장, 사설주차장)<br />\n※ 야영객 차량은 출입이 불가능, 휴양림...","상시개방(단, 시설 이용 시는 입, 퇴실시간 적용)<br>\n[숙박시설] 당일 15...","가족여행,걷기길,경상권,관광지,등산,등산로,비슬산,비슬산자연휴양림,여름방학가볼만한곳...",내국인 예약안내:홈페이지 예약,128.5129249,35.6900527
3,125407,경북,35,12,불정자연휴양림,레포츠와 휴식을 동시에 즐길 수 있는 불정자연휴양림,불정자연휴양림은 경북 문경의 불정동 마을 안쪽에 위치한 나트막한 봉우리인 수정봉 (...,,0,1484,1,https://cdn.visitkorea.or.kr/img/call?cmd=VIEW...,경상북도 문경시 불정길 180,불정자연휴양림 054-552-9443<br>\n문경관광진흥공단 054-553-310...,주차가능,[숙박시설]<br/>\n입실 14:00/퇴실 11:00,"관광지,불정자연휴양림,자연,자연휴양림,힐링여행","내국인 예약안내:인터넷 예약제이며, 당일 이용일 경우 전화예약만 가능<br />\n...",128.1342659,36.6188262
4,125408,경북,35,12,청송 자연휴양림 퇴적암층 (청송 국가지질공원),퇴적 구조를 관찰할 수 있는 학습장,청송자연휴양림 퇴적암층은 상쾌한 공기와 수려한 경관을 모두 갖춘 지질명소이다. 다양...,,4,1533,1,https://cdn.visitkorea.or.kr/img/call?cmd=VIEW...,경상북도 청송군 부남면 청송로 3478-96,청송군 문화관광과 054-870-6111<br>\n청송자연휴양림 054-872-3163,가능,09:00~18:00,"관광지,국가지질공원,자연,자연관광지,자연발생,자연속으로,자연여행,자연좋은곳,자연풍경...","시설이용료:[숲속의 집] 비수기 42,000원~84,000원 / 성수기 70,000...",129.0537206,36.3179194
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10633,3035809,경기,31,12,안산다문화음식거리,,안산역 1번 출구에서 약 600m 앞에는 안산다문화마을특구가 있다. 안산시는 세계 ...,,1,59,2,https://cdn.visitkorea.or.kr/img/call?cmd=VIEW...,경기도 안산시 단원구 원곡동,070-4740-2580(안산시관광안내소),가능(외국인주민센터 앞 공영주차장)<br>요금(기본 30분 300원 / 추가 10분...,매장별로 상이함,관광지,,126.7904142,37.3290507
10634,3035874,경기,31,12,대부해솔길 1코스,,대부해솔길은 해안선을 따라 대부도를 한 바퀴 둘러볼 수 있는 산책길로 91km에 이...,,0,33,0,https://cdn.visitkorea.or.kr/img/call?cmd=VIEW...,경기도 안산시 단원구 대부북동,1899-1720(대부도 관광안내소),가능<br>요금(무료),,레포츠,화장실:있음,126.5546122,37.2761527
10635,3036485,충남,34,12,월영산 출렁 다리,,"월영산 출렁다리는 월영산과 부엉산 사이를 잇는 높이 45m, 길이 275m, 폭 1...",,1,22,0,https://cdn.visitkorea.or.kr/img/call?cmd=VIEW...,충청남도 금산군 제원면 천내리,041-754-3837,가능,하절기(3월~10월) 09:00~18:00<br>\n동절기(11월~2월) 09:00...,관광지,입 장 료:무료,127.5975943,36.1152768
10636,3047124,전남,38,12,라테라스 윈터빌리지 어드벤처,라테라스의 따뜻한 겨울이야기,라테라스 윈터빌리지는 전라남도 여수시에 위치한 테마파크로 다양한 볼거리와 즐길 거리...,,0,81,2,https://cdn.visitkorea.or.kr/img/call?cmd=VIEW...,전라남도 여수시 돌산읍 진모1길 29-12,061-643-3500,가능,17:00~22:00,"겨울,겨울가볼만한곳,겨울방학갈만한곳,겨울에가볼만한곳,겨울여헹,관광지,라테라스,라테라...","시설이용료:[입장티켓]<br>\n- 주중(월~목) 12,900원<br>\n- 주말(...",127.7566755,34.7087802


In [8]:
df.columns

Index(['SPOT_ID', 'SPOT_CITY', 'SPOT_CITY_CODE', 'SPOT_CITY_CONTENT_TYPE',
       'SPOT_TITLE', 'SPOT_CATCHTITLE', 'SPOT_OVERVIEW', 'SPOT_TREATMENU',
       'SPOT_CONLIKE', 'SPOT_CONREAD', 'SPOT_CONSHARE', 'SPOT_IMGPATH',
       'SPOT_ADDR', 'SPOT_INFOCENTER', 'SPOT_PARKING', 'SPOT_USETIME',
       'SPOT_TAGNAME', 'SPOT_DETAIL', 'SPOT_LON', 'SPOT_LAT'],
      dtype='object')

In [1]:
import os
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import numpy as np
import pandas as pd
from konlpy.tag import Okt
from typing import List
from fastapi import FastAPI
from fastapi.responses import JSONResponse
from pydantic import BaseModel
import os
import time
import random
import dijkstra
import pymysql


# 둘중 하나만 사용

# file path
###################################################################################

current_dir = os.getcwd()
# df_path = os.path.join(current_dir, 'travel_spot_v1.csv') sql로 변경

model_path = os.path.join(current_dir, 'travel_model_v5.pkl')
info_path = os.path.join(current_dir, 'DB_INFO.pkl')

###################################################################################

with open(info_path, "rb") as file:
    USER = pickle.load(file)
    PASSWD = pickle.load(file)
    HOST = pickle.load(file)
    PORT = pickle.load(file)
    NAME = pickle.load(file)

# pymysql로 dataFrame 호출

host = HOST
user = USER
password = PASSWD
database = NAME
charset = 'utf8mb4'

conn = pymysql.connect(host=host,user=user,password=password,database=database,charset=charset)
sql = "select * from TRAVEL_SPOT"
cursor = conn.cursor()
cursor.execute(sql)
df = pd.DataFrame(cursor.fetchall(), columns= [i[0] for i in cursor.description])



In [2]:

# db.to_csv('df.csv')
# df = pd.read_csv('./df.csv', index_col=0)


okt = Okt()
app = FastAPI()
df = df.fillna('')

spot_df = df[df['SPOT_CITY_CONTENT_TYPE'] != 39]
food_df = df[df['SPOT_CITY_CONTENT_TYPE'] == 39]



In [3]:

with open(model_path, 'rb') as file:
    loaded_data = pickle.load(file)
    
    vectorizer_spot = loaded_data["spot"]["vectorizer"]
    matrix_spot = loaded_data["spot"]["matrix"]
    city_matrices_spot = loaded_data["spot"]["city_matrices"]

    vectorizer_food = loaded_data["food"]["vectorizer"]
    matrix_food = loaded_data["food"]["matrix"]
    city_matrices_food = loaded_data["food"]["city_matrices"]
    


In [4]:


def add_to_similar_tags(df, sorted_indices, cos_similarities, similar_tags):
    bag = []
    for index in sorted_indices:
        bag.append({
            "id": str(df.iloc[index]['SPOT_ID']),
            "area": str(df.iloc[index]['SPOT_CITY']),
            "title": str(df.iloc[index]['SPOT_TITLE']),
            "similarity": str(("{:.1f}".format(cos_similarities[0][index]*100))) + '%',
            "catchtitle": str(df.iloc[index]['SPOT_CATCHTITLE']),
            "detail": str(df.iloc[index]['SPOT_DETAIL']),
            "treatMenu": str(df.iloc[index]['SPOT_TREATMENU']),
            "tagName": str(df.iloc[index]['SPOT_TAGNAME']),
            "addr": str(df.iloc[index]['SPOT_ADDR']),
            "info": str(df.iloc[index]['SPOT_INFOCENTER']),
            # "lat": str(df.iloc[index]['parking']),
            "useTime": str(df.iloc[index]['SPOT_USETIME']),
            "conLike": str(df.iloc[index]['SPOT_CONLIKE']),
            "conRead": str(df.iloc[index]['SPOT_CONREAD']),
            "conShare": str(df.iloc[index]['SPOT_CONSHARE']),
            "imgPath": str(df.iloc[index]['SPOT_IMGPATH']),
            "overView": str(df.iloc[index]['SPOT_OVERVIEW'][:100]),
            "lat": str(df.iloc[index]['SPOT_LAT']),
            "lon": str(df.iloc[index]['SPOT_LON'])
            })
    similar_tags.append(bag)




In [7]:





area = '서울'
question_ = '피자 치킨'

# 미리 계산된 해당 지역의 TF-IDF 행렬 사용
city_spot = city_matrices_spot[f'{area}']
city_food = city_matrices_food[f'{area}']

food_city_df = food_df[food_df['SPOT_CITY']== f'{area}']
spot_city_df = spot_df[spot_df['SPOT_CITY']== f'{area}']


# 질문과 선택된 지역의 TF-IDF로 유사도 계산
question_spot = vectorizer_spot.transform([' '.join(okt.nouns(f'{question_}'))])
question_food = vectorizer_food.transform([' '.join(okt.nouns(f'{question_}'))])

#spot 인덱스 추출
cos_similarities_spot = linear_kernel(question_spot, city_spot)
sorted_indices_spot = np.argsort(cos_similarities_spot[0])[::-1][:5]

#food 인덱스 추출
cos_similarities_food = linear_kernel(question_food, city_food)
sorted_indices_food = np.argsort(cos_similarities_food[0])[::-1][:5]
print([spot_df['SPOT_CITY']== f'{area}'][0])

print('df = ',len(df))
print('spot = ',len(spot_df))
print('food = ',len(food_df))
print('---------------------------')
print('spot_city_df = ',len(spot_city_df))
print('food_city_df = ',len(food_city_df))
print('cos_similarities_spot = ',len(cos_similarities_spot[0]))
print('cos_similarities_food = ',len(cos_similarities_food[0]))
print('---------------------------')
    
# 결과를 저장할 리스트 초기화
similar_tags = []

# 리스트로 추가
try:
    add_to_similar_tags(spot_city_df, sorted_indices_spot, cos_similarities_spot, similar_tags)
    add_to_similar_tags(food_city_df, sorted_indices_food, cos_similarities_food, similar_tags)
    print('done')

except Exception as e:
    print(e)
    print('not added')
print(len(similar_tags))


try:
    st0, st1 = similar_tags[0], similar_tags[1]
    travel_order, travel_distances = dijkstra.dijkstra(st0, start=0)
    food_order, food_distances = dijkstra.dijkstra(st1, start=0)
    
    st0 = [st0[order] for order in travel_order]
    st1 = [st1[order] for order in food_order]
    
    st = [st0, st1]
except Exception as e:
    st = []
    print(e)
    print(similar_tags)
    
print(similar_tags)






0        False
1        False
2        False
3        False
4        False
         ...  
10633    False
10634    False
10635    False
10636    False
10637    False
Name: SPOT_CITY, Length: 4708, dtype: bool
df =  10638
spot =  4708
food =  5930
---------------------------
spot_city_df =  166
food_city_df =  205
cos_similarities_spot =  166
cos_similarities_food =  205
---------------------------
done
2
[[{'id': '3023349', 'area': '서울', 'title': '돈덕전', 'similarity': '0.0%', 'catchtitle': '100년 만에 복원된 대한제국의 영빈관', 'detail': '입 장 료:무료<br>\n※ 단, 덕수궁 입장권을 별도로 구입', 'treatMenu': '', 'tagName': '가볼래터,가족과함께,관광지,대한제국,덕수궁,서울가볼만한곳,아이와함께,여행구독,연인과함께,영빈관,전시', 'addr': '서울특별시 중구 세종대로 99 (정동)', 'info': '02-771-9951', 'useTime': '09:00~17:30 <br>\n※ 입장마감 17:00', 'conLike': '6', 'conRead': '521', 'conShare': '4', 'imgPath': 'https://cdn.visitkorea.or.kr/img/call?cmd=VIEW&id=30f2dbf8-42cb-4919-9fe4-7f19c569376c', 'overView': '대한제국 외교 공간으로 기획되었던 돈덕전은 덕수궁 내에 있는 서양식 2층 건물이다. ‘돈덕(惇德)’의 뜻은 ‘덕 있는 이를 도탑게 해 어진 이를 